In [6]:
from ipywidgets import Layout, Button, Box, Label, Text, Output, Tab
from IPython.display import display
from timeStamp import rfidTimeStamp as rfidModule
import sqlite3 

class timeRegistration(rfidModule):
    '''
        timeRegistration is a class that contains the UI. Functionality:
        
        1) Create new users in the data base
        2) Modify time registration of existing users. This function is made to allow write into the database should the user
        accidentially forget to log in or log out
        3) Users time registration data can be extracted within time period and allow report generation followed by sending report over email. 
    '''
    def __init__(self, dbPath):
        
        #Connect to the database
        self.dbPath = dbPath
        self.createDatabaseIfNotExist(dbPath)
        
        #Initializing the GUI
        self.buildGUI()
    
    def buildGUI(self):
        
        #REGISTER NEW EMPLOYEE
        form_item_layout = Layout(
        display = 'flex',
        flex_flow = 'row',
        justify_content = 'space-between')
        
        self.employeeRegNameText = Text(placeholder = 'Employee full name', disabled = False)
        self.employeeRegButton = Button(description = 'Register', disabled = False, 
                                        button_style = 'success',
                                        tooltip = 'Register new employee card')
        
        self.employeeRegButton.on_click(self.registerNewEmployee)
        
        self.employeeRegOut = Output()
        
        self.employeeRegFormItems = [
            Box([Label(value = 'New Employee name : '), self.employeeRegNameText]),
            Box([self.employeeRegButton]),
            Box([self.employeeRegOut])
        ]
        
        self.registerForm = Box(self.employeeRegFormItems, layout = Layout(
        display = 'flex',
        flex_flow = 'column',
        border = 'solid 2px',
        align_items = 'stretch',
        width = '70%'))
        
        #TAB
        self.tab = Tab()
        self.tab.children = [self.registerForm]
        self.tab.set_title(0, 'Register')
        
        display(self.tab)
        
    def createDatabaseIfNotExist(self, databasePath):
        '''
        Check if the table of the data base provided by the databasePath exists. If not it will be created.
        '''
        conn = sqlite3.connect(databasePath)
        sqlString = '''CREATE TABLE IF NOT EXISTS workinghour (name text, employeeNo text, datetime text, status text)'''
        c = conn.cursor()
        c.execute(sqlString)
        conn.commit()
        conn.close()
    
    def registerNewEmployee(self, change):
        #Extract employee full name from the text
        employeeName = self.employeeRegNameText.value
        if not employeeName:
            with self.employeeRegOut:
                self.employeeRegOut.clear_output()
                print('Employee name is empty and therefore cannot register new employee')
                return

        with self.employeeRegOut:
            self.employeeRegOut.clear_output()
            print('Ready to register ' + employeeName + ' swipe the card to the reader')
            self.writeCard(employeeName)
            print('Success!')

obj = timeRegistration('employeeHour.db')

In [4]:
import sqlite3
databasePath = 'employeeWorkingHour.db'
conn = sqlite3.connect(databasePath)
sqlString = '''CREATE TABLE IF NOT EXISTS workinghour (name text, employeeNo text, datetime text, status text)'''
c = conn.cursor()
c.execute(sqlString)
conn.commit()
conn.close()